In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast, os, re, json, requests
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''

###1-1. extract text data

In [ ]:
txts = glob(data_folder+'data/raw_txt/*')
len(txts)

1603

In [ ]:
!mkdir $data_folder'data/raw_json'

In [ ]:
titles = ['1. NAME OF THE MEDICINAL PRODUCT','2. QUALITATIVE AND QUANTITATIVE COMPOSITION', '3. PHARMACEUTICAL FORM', '4. CLINICAL PARTICULARS',
          '5. PHARMACOLOGICAL PROPERTIES', '6. PHARMACEUTICAL PARTICULARS', '7. MARKETING AUTHORISATION HOLDER', '8. MARKETING AUTHORISATION NUMBER',
          '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION','10. DATE OF REVISION OF THE TEXT']

In [ ]:
not_found = []
not_found_next = []
for txt in tqdm(txts):
  drug = txt.split('/')[-1].replace('.txt','')
  with open(txt, 'r+') as fi:
    txt = fi.read()
    txt = txt.split('ANNEX II')[0]
  #we're interested in "4.CLINICAL PARTICULARS". extract this section
  if '4. CLINICAL' in txt:
    if '5. PHARMACOLOGICAL' in txt:
      sec_dict = {}
      sec_dict['4. CLINICAL PARTICULARS'] = '4. CLINICAL'+txt.split('4. CLINICAL')[1].split('5. PHARMACOLOGICAL')[0]+'5. PHARMACOLOGICAL'
      with open(data_folder+'data/raw_json/{}.json'.format(drug), "w") as out:
        json.dump(sec_dict, out)
    elif '6. PHARMACEUTICAL' in txt:
      sec_dict = {}
      sec_dict['4. CLINICAL PARTICULARS'] = '4. CLINICAL'+txt.split('4. CLINICAL')[1].split('6. PHARMACEUTICAL')[0]+'6. PHARMACEUTICAL'
      with open(data_folder+'data/raw_json/{}.json'.format(drug), "w") as out:
        json.dump(sec_dict, out)
    else:
      not_found_next.append(txt)
  else:
    if '4.1 Therapeutic' in txt:
      sec_dict = {}
      sec_dict['4. CLINICAL PARTICULARS'] = '4. CLINICAL PARTICULARS'+txt.split('4.1 Therapeutic')[1].split('5. PHARMACOLOGICAL')[0]+'5. PHARMACOLOGICAL'
      with open(data_folder+'data/raw_json/{}.json'.format(drug), "w") as out:
        json.dump(sec_dict, out)
    elif '4. CLINICAL' in txt.replace('\n',''):
      sec_dict = {}
      sec_dict['4. CLINICAL PARTICULARS'] = '4. CLINICAL'+txt.replace('\n','').split('4. CLINICAL')[1].split('5. PHARMACOLOGICAL')[0]+'5. PHARMACOLOGICAL'
      with open(data_folder+'data/raw_json/{}.json'.format(drug), "w") as out:
        json.dump(sec_dict, out)
    elif '4. CLINICAL' in ' '.join(txt.split()):
      sec_dict = {}
      sec_dict['4. CLINICAL PARTICULARS'] = '4. CLINICAL'+' '.join(txt.split()).split('4. CLINICAL')[1].split('5. PHARMACOLOGICAL')[0]+'5. PHARMACOLOGICAL'
      with open(data_folder+'data/raw_json/{}.json'.format(drug), "w") as out:
        json.dump(sec_dict, out)
    else:
      not_found.append(txt)
len(not_found), len(not_found_next)

100%|██████████| 1603/1603 [00:14<00:00, 114.03it/s]


(13, 18)

In [ ]:
len(glob(data_folder+'data/raw_json/*'))

1590

---

In [1]:
!mkdir $data_folder'data/raw_json_ue'

mkdir: cannot create directory ‘data/raw_json_ue’: No such file or directory


In [ ]:
### extract sub-sections from section 4. CLINICAL PARTICULARS
sec4_titles = ['4.1 Therapeutic indications', '4.2 Posology and method of administration', '4.3 Contraindications', '4.4 Special warnings and precautions for use',
               '4.5 Interaction with other medicinal products and other forms of interaction', '4.6 Fertility, pregnancy and lactation', '4.7 Effects on ability to drive and use machines',
               '4.8 Undesirable effects', '4.9 Overdose']
jsons = glob(data_folder+'data/raw_json/*')
not_found_ue = []
for j in tqdm(jsons):
  with open(j, 'r+') as fi:
    d = orjson.loads(fi.read())
  try:
    txt = d['4. CLINICAL PARTICULARS']
    txt = ' '.join(txt.split())
    ue = txt.split('4.8 Undesirable')[1].split('4.9 Overdose')[0]
    ue_dict = {}
    ue_dict['Undesirable effects'] = ue
    drug = j.split('/')[-1].replace('.json','')
    with open(data_folder+'data/raw_json_ue/{}.json'.format(drug), "w") as out:
        json.dump(ue_dict, out)
  except:
    not_found_ue.append(json)
len(not_found_ue)

100%|██████████| 1590/1590 [00:14<00:00, 110.47it/s]


78

In [ ]:
### extract sub-sections from section 4. CLINICAL PARTICULARS
sec4_titles = ['4.1 Therapeutic indications', '4.2 Posology and method of administration', '4.3 Contraindications', '4.4 Special warnings and precautions for use',
               '4.5 Interaction with other medicinal products and other forms of interaction', '4.6 Fertility, pregnancy and lactation', '4.7 Effects on ability to drive and use machines',
               '4.8 Undesirable effects', '4.9 Overdose']
jsons = glob(data_folder+'data/raw_json/*')
not_found_ue = []
for j in tqdm(jsons):
  with open(j, 'r+') as fi:
    d = orjson.loads(fi.read())
  try:
    txt = d['4. CLINICAL PARTICULARS']
    txt = ' '.join(txt.split())
    ue = txt.split('4.4 Special')[1].split('4.5 Interaction')[0]
    ue_dict = {}
    ue_dict['Special warnings'] = ue
    drug = j.split('/')[-1].replace('.json','')
    with open(data_folder+'data/raw_json_ue/{}.json'.format(drug), "w") as out:
        json.dump(ue_dict, out)
  except:
    not_found_ue.append(json)
len(not_found_ue)

---

In [ ]:
jsons = glob(data_folder+'data/raw_json_ue/*')
len(jsons)

1512

In [ ]:
ade_list = []
for j in tqdm(jsons):
  with open(j, 'r+') as fi:
    d = orjson.loads(fi.read())
  ade = d['Undesirable effects']
  pattern = r'[^\n\w\s.,]'
  clean_ade = re.sub(r'[^\n\w\s.,]', '', ade)
  drug = j.split('/')[-1].replace('.json','')
  ade_list.append([drug, clean_ade])

100%|██████████| 1512/1512 [00:02<00:00, 551.37it/s]


In [ ]:
ade_text_table_df = pd.DataFrame(ade_list, columns=['drug', 'ade_text'])
ade_text_table_df.to_csv(data_folder+'data/ade_text_table.csv', index=False)
ade_text_table_df.head(1)

,drug,ade_text
0,extavia,e ffects Summary of the safety profile At the...


In [ ]:
ade_text_table_df.shape

(1512, 2)

###2-1. extract tabular data

In [ ]:
tbls = glob(data_folder+'data/raw_tbl/*')

In [ ]:
ade_tbl = []
for tbl in tqdm(tbls):
  drug = tbl.split('/')[-1].split('_')[0]
  try:
    df = pd.read_csv(tbl)
    if any([i in list(df.columns) for i in ['System', 'Very common', 'Common', 'Not known', 'Rare', 'Uncommon']]):
      col = df.columns.tolist()
      for i in col:
        l = ' '.join([i for i in df[i].tolist() if str(i) != 'nan'])
        ade_tbl.append([drug, i, l])
  except:
    pass

100%|██████████| 138910/138910 [5:22:37<00:00,  7.18it/s]


In [ ]:
ades_df = pd.DataFrame(ade_tbl, columns = ['drug', 'col', 'txt'])
ades_df.to_csv(data_folder+'data/raw_ade_table.csv', index=False)
ades_df.head(1)

,drug,col,txt
0,nexavar,Unnamed: 0,Hepatobi- liary disorders hepatitis* Skin and ...
